# Рекомендация тарифов

# Исследование и выбор оптимального тарифа мобильной связи



Мы будем анализировать информацию о клиентах мобильной сети "Мегалайн", использующих архивные тарифы, с целью предложить им новые тарифы «Смарт» или «Ультра». Для того, чтобы предложить клиентам оптимальный тариф, построим модель на основе данных о поведении клиентов, которые уже перешли на новые тарифы.

## Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from joblib import dump

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
display(data)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
data.info() #посмотрим тип данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data['messages']=data['messages'].astype("int")  #изменим тип столбца с количеством сообщений на тип int, так как по сути это целые числа
#data['messages'].dtype
data['calls']=data['calls'].astype("int")  #изменим тип столбца с количеством звонков на тип int, так как по сути это целые числа
data['calls'].dtype

dtype('int64')

In [5]:
 data.duplicated().sum()  #проверка явных дубликатов

0

# Вывод:
а. при первичном рассмотрении можно отметить, что файл имеет 3214 строк и 5 столбцов;

б. мы изменили типы столбцов с количеством сообщений и количеством звонков на int, так как значения этих столбцов - целые числа;

в. явных дубликатов нет;

г. пропущенных значений нет.


## Разобьем данные на выборки

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую (обычно делят на 60, 20 и 20% соответственно).

2.1. Разбиваем исходные данные на обучающую(60%) и валидационную плюс тестовую выборки(в совокупности 40%).

In [6]:
data_train, data_valtest = train_test_split(data, test_size=0.4, random_state=12345)
features = data_valtest.drop('is_ultra', axis=1)
target = data_valtest['is_ultra']

features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


2.2. Разбиваем полученные выше данные на валидационную и тестовую выборки (по 20 % от изначальных данных, по 50% от данных по валидационной и тестовой выборке в совокупности).

In [7]:
features_valid, features_test, target_valid, target_test = train_test_split(features, target, 
                                                                            test_size=0.5, random_state=12345)

print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)
(643, 4)
(643,)


# Вывод:
мы разбили исходные данные на обучающую(60%), валидационную (20%) и тестовую (20%) выборки.

## Исследуем модели

3.1. Исследуем модель дерева решений.

In [8]:
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth) # обучите модель с заданной глубиной дерева
    model.fit(features_train,target_train) # обучите модель
    predictions_valid = model.predict(features_valid) # получите предсказания модели
    result = accuracy_score(target_valid, predictions_valid) # посчитайте качество модели
    print('max_depth =', depth, ':', result)
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Лучшая глубина дерева=", best_depth, ";лучшее качество=", best_accuracy)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
Лучшая глубина дерева= 3 ;лучшее качество= 0.7853810264385692


3.2. Исследуем модель случайного леса

In [9]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(10,30):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
                 
print( "Количество деревьев:", best_est, "глубина дерева:", best_depth,"качество:", best_accuracy)


Количество деревьев: 26 глубина дерева: 7 качество: 0.807153965785381


3.3. Исследуем модель логистической регрессии

In [10]:
model = LogisticRegression(solver='lbfgs')
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество:", accuracy)


Качество: 0.7107309486780715


# Вывод:
а. лучшее качество у модели дерева решений (DecisionTreeClassifier) - 0.7853810264385692, лучшая глубина дерева - 3;

б. лучшее качество у модели случайного леса (RandomForestClassifier) - 0.807153965785381, количество деревьев - 26, глубина дерева - 7;

в. результат Accuracy у модели логистической регрессии (LogisticRegression) - 0.7107309486780715.

## Проверим модель на тестовой выборке

Проверим на тестовой выборке модель, которая показала себя лучшим образом - модель случайного леса (RandomForestClassifier)

In [11]:
model = RandomForestClassifier(max_depth=7, n_estimators=26, random_state=12345)
model.fit(features_train, target_train)
accuracy_test = model.score(features_test, target_test)
print("Качество:", accuracy_test)


Качество: 0.80248833592535


# Вывод:

Качество модели случайного леса (RandomForestClassifier) на тестовой выборке - 0.80248833592535.

## Проверим модели на адекватность

5.1. Создадим простейшую (dummy) модель для получения контрольных данных для сравнительной оценки построенной нами более сложной модели и определим ее качество.

In [12]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Качество DummyClassifier:', result_dc)

Качество DummyClassifier: 0.7060653188180405


5.2. Выведем качество нашей лучшей модели.

In [13]:
print('Качество RandomForestClassifier:',accuracy_test )

Качество RandomForestClassifier: 0.80248833592535


# Вывод:
наша модель адекватна, так как ее качество выше качества контрольной модели.

# Общий вывод:

1. у нас чистые исходные данные без пропусков и дубликатов;

2. при исследовании нескольких моделей (DecisionTreeClassifier, RandomForestClassifier, LogisticRegression) наилучшее качество показала модель RandomForestClassifier -  0.807153965785381 с количеством деревьев - 26 и глубиной дерева - 7;

3. качество модели случайного леса (RandomForestClassifier) на тестовой выборке - 0.80248833592535.

4. наша модель RandomForestClassifier адекватна, что выявила проверка, сравнение с контрольной моделью DummyClassifier, качество DummyClassifier - 0.7060653188180405.
